# Batch Processing

Learn how to efficiently process multiple prompts and handle large-scale operations.

## What You'll Learn
- Processing multiple inputs efficiently
- Batch API operations
- Result aggregation and analysis
- Error handling strategies

In [ ]:
from prompt_playground.client import create_client, send_batch
from prompt_playground.analysis import compare_responses
from prompt_playground.cache import ResponseCache
from rich import print as rprint
import pandas as pd

client = create_client()
cache = ResponseCache(ttl=3600)
rprint('[green]✓[/green] Setup complete')

## Basic Batch Processing

In [ ]:
topics = ['Python', 'JavaScript', 'Rust', 'Go', 'TypeScript']
prompts = [f'Explain {topic} in one sentence.' for topic in topics]

responses = send_batch(prompts=prompts, client=client)

for topic, response in zip(topics, responses):
    rprint(f'\n[cyan]{topic}:[/cyan] {response["text"]}')
    rprint(f'Tokens: {response["output_tokens"]}')

## Using Templates for Batch Operations

In [ ]:
from prompt_playground.prompts import PromptTemplate

template = PromptTemplate(
    template='Translate "{text}" to {language}.',
    variables=['text', 'language']
)

translations = [
    {'text': 'Hello world', 'language': 'Spanish'},
    {'text': 'Hello world', 'language': 'French'},
    {'text': 'Hello world', 'language': 'German'},
]

prompts = [template.fill(**t) for t in translations]
responses = send_batch(prompts=prompts, temperature=0.3, client=client)

for trans, resp in zip(translations, responses):
    rprint(f'{trans["language"]}: {resp["text"]}')

## Caching for Efficiency

In [ ]:
import time

prompt = 'What is machine learning?'
model = 'claude-sonnet-4-5-20250929'
params = {'temperature': 1.0, 'max_tokens': 100}

start = time.time()
cached = cache.get(prompt, model, params)
if cached:
    rprint('[green]Cache hit![/green]')
else:
    from prompt_playground.client import send_prompt
    response = send_prompt(prompt=prompt, **params, client=client)
    cache.set(prompt, model, params, response)
    rprint('[yellow]Cache miss - stored[/yellow]')
end = time.time()

rprint(f'Time: {(end-start)*1000:.2f}ms')
stats = cache.get_stats()
rprint(f'Hit rate: {stats["hit_rate_percent"]}%')

## Error Handling in Batches

In [ ]:
def process_batch_safe(prompts, client):
    results = []
    errors = []
    
    for i, prompt in enumerate(prompts):
        try:
            response = send_prompt(prompt=prompt, client=client)
            results.append({'index': i, 'success': True, 'response': response})
        except Exception as e:
            errors.append({'index': i, 'error': str(e)})
            results.append({'index': i, 'success': False, 'error': str(e)})
    
    return results, errors

test_prompts = ['Valid prompt', 'Another valid one']
results, errors = process_batch_safe(test_prompts, client)

rprint(f'Success: {sum(1 for r in results if r["success"])}/{len(prompts)}')
rprint(f'Errors: {len(errors)}')

## Aggregating Results

In [ ]:
prompts = [f'Write a {word} sentence.' for word in ['short', 'medium', 'long']]
responses = send_batch(prompts=prompts, client=client)

df = compare_responses(responses)
df['type'] = ['short', 'medium', 'long']

rprint('\n[bold]Summary:[/bold]')
rprint(f'Total tokens: {df["total_tokens"].sum()}')
rprint(f'Total cost: ${df["estimated_cost"].sum():.6f}')
rprint(f'Avg words: {df["word_count"].mean():.1f}')

display(df[['type', 'word_count', 'output_tokens', 'estimated_cost']])

## Best Practices

### 1. Use Appropriate Batch Sizes

In [ ]:
def process_in_chunks(prompts, chunk_size=10):
    results = []
    for i in range(0, len(prompts), chunk_size):
        chunk = prompts[i:i+chunk_size]
        responses = send_batch(prompts=chunk, client=client)
        results.extend(responses)
        rprint(f'Processed {min(i+chunk_size, len(prompts))}/{len(prompts)}')
    return results

rprint('[green]✓[/green] Chunk processing prevents timeouts')

### 2. Monitor Costs

In [ ]:
def batch_with_budget(prompts, max_cost, client):
    total_cost = 0
    results = []
    
    for prompt in prompts:
        if total_cost >= max_cost:
            rprint(f'[yellow]Budget limit reached: ${total_cost:.6f}[/yellow]')
            break
        
        response = send_prompt(prompt=prompt, client=client)
        from prompt_playground.client import estimate_cost
        cost = estimate_cost(response['input_tokens'], response['output_tokens'], response['model'])
        total_cost += cost['total_cost']
        results.append(response)
    
    rprint(f'[green]Processed {len(results)} prompts for ${total_cost:.6f}[/green]')
    return results

## Summary

You've learned:
- ✓ Batch processing basics
- ✓ Using templates for scale
- ✓ Caching for efficiency
- ✓ Error handling strategies
- ✓ Result aggregation
- ✓ Cost management

## Next Steps

- **05_evaluation_metrics.ipynb**: Evaluate batch results